In [1]:
import add_path
import datasets
import random
import json

d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from models.BERT.tucoregcn_bert_pytorch_processor import DialogRE
dev = DialogRE()
gen = dev._generate_examples("../datasets/DialogRE/dev.json", "dev")


In [3]:
test = gen.send(None)[1]
test

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


{'dialog': "{speaker_y} hey!\n{speaker_2} hey.\n{speaker_3} hey, man. what's up?\n{speaker_y} maybe you can tell me. my agent would like to know why i didn't show up at the audition i didn't know i had today. the first good thing she gets me in weeks. how could you not give me the message?!\n{speaker_3} well, i'll tell ya i do enjoy guilt, but, ah, it wasn't me.\n{speaker_2} yes, it was! it was him! uh huh! okay, it was me!\n{speaker_y} how is it you?\n{speaker_2} well, it was just, it was all so crazy, you know. i mean, chandler was in the closet, counting to 10, and he was up to 7 and i hadn't found a place to hide yet. i-i-i meant to tell you, and i wrote it all down on my hand. see, all of it.\n{speaker_y} yep, that's my audition.\n{speaker_4} see, now this is why i keep notepads everywhere.\n{speaker_2} yep, and that's why we don't invite you to play.\n{speaker_5} what is the great tragedy here? you go get yourself another appointment.\n{speaker_y} well, estelle tried, you know. t

In [4]:
from transformers.models.bert.tokenization_bert import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [6]:
from models.BERT.speaker_tokens import SPEAKER_TOKENS, SpeakerBertTokenizer
speaker_tokenizer = SpeakerBertTokenizer.from_pretrained('bert-base-uncased')
speaker_tokens = [entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not x[1][0].startswith("__") else None, enumerate(SPEAKER_TOKENS.__dict__.items())))]
tokens = speaker_tokenizer.tokenize(test['dialog'])
speaker_x = speaker_tokenizer.tokenize(test['relation'].speaker_x)
speaker_y = speaker_tokenizer.tokenize(test['relation'].speaker_y)
print(" ".join(tokens))
sequence = ["CLS"] + tokens + ["SEP"] + speaker_x + ["SEP"] + speaker_y + ["SEP"]
input_ids = speaker_tokenizer.convert_tokens_to_ids(sequence)
input_mask = [1] + [1]*len(tokens) + [1] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [0]
segment_ids = [0] + [0]*len(tokens) + [0] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [1]
input_speaker_ids = []
current_speaker_id = 0
for token in sequence:
	if SPEAKER_TOKENS.is_speaker(token):
		current_speaker_id = SPEAKER_TOKENS.convert_speaker_to_id(token)
	input_speaker_ids.append(current_speaker_id)
speaker_ids = [0] + input_speaker_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]
print(f"Length: {len(input_ids)}, "+" ".join([str(input) for input in input_ids]))
print(f"Length: {len(input_mask)}, "+" ".join([str(mask) for mask in input_mask]))
print(f"Length: {len(segment_ids)}, "+" ".join([str(segment) for segment in segment_ids]))
print(f"Length: {len(speaker_ids)}, "+" ".join([str(speaker) for speaker in speaker_ids]))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


{speaker_y} hey ! {speaker_2} hey . {speaker_3} hey , man . what ' s up ? {speaker_y} maybe you can tell me . my agent would like to know why i didn ' t show up at the audition i didn ' t know i had today . the first good thing she gets me in weeks . how could you not give me the message ? ! {speaker_3} well , i ' ll tell ya i do enjoy guilt , but , ah , it wasn ' t me . {speaker_2} yes , it was ! it was him ! uh huh ! okay , it was me ! {speaker_y} how is it you ? {speaker_2} well , it was just , it was all so crazy , you know . i mean , chandler was in the closet , counting to 10 , and he was up to 7 and i hadn ' t found a place to hide yet . i - i - i meant to tell you , and i wrote it all down on my hand . see , all of it . {speaker_y} yep , that ' s my audition . {speaker_4} see , now this is why i keep note ##pad ##s everywhere . {speaker_2} yep , and that ' s why we don ' t invite you to play . {speaker_5} what is the great tragedy here ? you go get yourself another appointment 